In [1]:
import ast
import sys

'''
FILENAME = sys.argv[1]
min_idx = int( sys.argv[2] )
max_idx = int( sys.argv[3] )
MODELS = [sys.argv[4]]
CURVES = [sys.argv[5]]
RESULTS = ast.literal_eval( sys.argv[6] )
'''

FILENAME = 'OBESIDAD'
min_idx = 1
max_idx = 5

MODELS = [
        #'xgb-n1-cregex',
        'nb-n1-cregex',
        #'svm-n1-cregex',
        #'rf-n1-cregex'
         ]

CURVES = ['PL', 'AL']

RESULTS = True

if RESULTS:
    #MODELS = [m for m in MODELS if 'regex' in m]
    CURVES = ['RESULTS']

import warnings
warnings.filterwarnings("ignore")
import logging
logging.captureWarnings(True)
logging.disable(sys.maxsize)
from cregex import *
from utils import *
from curves import *
seed_everything()

N_CLASSES = {'FUMADOR':2, 'OBESIDAD':2, 'OBESIDAD_TIPOS':3}[FILENAME]
HYPERPARAMS['bert']['n_classes'] = N_CLASSES
HYPERPARAMS['setfit']['n_classes'] = N_CLASSES
HYPERPARAMS['zsl']['n_classes'] = N_CLASSES
HYPERPARAMS['zsl']['labels'] = labels[FILENAME]

create_paths(FILENAME)

with open( os.path.join( os.getcwd(), 'snippets_procesados_'+FILENAME),  'rb') as a:
    data = pickle.load(a)
    data = sorted(data, key = lambda x:x[0], reverse = False)
    DATA = np.array( [snippet for snippet, classe in data] )#[:50]
    CLASSES = np.array( [classe for snippet, classe in data])#[:50]

print(FILENAME)
RUNS = 1
FOLDS = 5
folds = KFold(n_splits = FOLDS, shuffle = False)
idxs = np.arange(0, len(DATA))

for r in range(RUNS):
    idxs = shuffle(idxs, random_state = SEED)
    CLASSES = CLASSES[idxs]
    DATA = DATA[idxs]
    k = -1
    for train_index, test_index in folds.split(idxs):
        k+=1
        print('fold:', k+1)
        if (k+1) not in list(range(min_idx, max_idx+1)):
            continue

        for CURVE in CURVES:

            for MODEL in MODELS:

                if 'PL' in CURVE and 'regex' in MODEL:
                    continue

                print('CURVE:', CURVE)
                print('MODEL:', MODEL)

                X_train = copy.deepcopy( DATA[train_index] )
                y_train = copy.deepcopy( CLASSES[train_index] )
                X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=SEED)
                X_test = copy.deepcopy( DATA[test_index] )
                y_test = copy.deepcopy( CLASSES[test_index] )

                curve = Curves(
                    X_train, y_train,
                    X_val, y_val,
                    X_test,
                    N_CLASSES, CURVE, MODEL,
                    BATCH, FILENAME
                )

                if not RESULTS:
                    curve.learningCurve()
                    results = [
                               curve.results['scores'],
                               curve.results['x'],
                               curve.results['y'],
                               curve.results['y_u_dst'],
                               curve.results['y_clf'],
                               curve.results['dst_cregex'],
                               curve.HYPERPARAMS,        
                               curve.N_FEATURES,        
                               y_test
                    ]
                    with open( os.path.join( os.getcwd(), 'out', 'RESULTSLC', CURVE, FILENAME, FILENAME+'_'+MODEL+'_'+CURVE+'_k'+str(k+1)+'.pkl' ), 'wb' ) as a:
                        pickle.dump(results, a, protocol=2)

                else: #results
                    pred_aux, _, __, model = curve.model_selection(X_train, y_train, X_test, X_u=[], results=False, return_model=True)
                    results_y = copy.deepcopy(model.y)
                    dst_y = copy.deepcopy(model.distribution)
                    for key in results_y:
                        pred = results_y[key]
                        if 'regex' in key:
                            times = [copy.deepcopy(model.times['train-'+'cregex']),
                                     copy.deepcopy(model.times['predict-'+key])
                                    ]
                        else:
                            times = [copy.deepcopy(model.times['train-'+key]),
                                     copy.deepcopy(model.times['predict-'+key])
                                    ]
                        with open( os.path.join( os.getcwd(), 'out', 'RESULTS', FILENAME, FILENAME+'_'+key+'_'+'RESULTS'+'_k'+str(k+1)+'.pkl' ), 'wb' ) as a:
                            pickle.dump([pred, times, dst_y, y_train, y_test,
                                         X_train, X_test], a, protocol=2)
                    del model
                    del pred_aux
                    del results_y
                    del pred
                    del times
                    del dst_y
                    gc.collect()

                del X_train
                del X_val
                del X_test
                del y_train
                del y_val
                del y_test
                gc.collect()

OBESIDAD
fold: 1
CURVE: RESULTS
MODEL: nb-n1-cregex
PATH-fregex... G:\My Drive\Research\UOH\BHI2024
sw_cpp_OBESIDAD.exe was removed
CREGEX...fit
PATH-fregex... G:\My Drive\Research\UOH\BHI2024
TOKENS_OBESIDAD.txt was removed
DATOSX_OBESIDAD.txt was removed
CORPUSX_OBESIDAD.txt was removed
CLASESX_OBESIDAD.txt was removed
sw_cpp_OBESIDAD.exe was removed
nb-n1...fit
nb-n1...predict
CREGEX...predict
fold: 2
CURVE: RESULTS
MODEL: nb-n1-cregex
PATH-fregex... G:\My Drive\Research\UOH\BHI2024
TOKENS_OBESIDAD.txt was removed
DATOSX_OBESIDAD.txt was removed
CORPUSX_OBESIDAD.txt was removed
CLASESX_OBESIDAD.txt was removed
sw_cpp_OBESIDAD.exe was removed
CREGEX...fit
PATH-fregex... G:\My Drive\Research\UOH\BHI2024
TOKENS_OBESIDAD.txt was removed
DATOSX_OBESIDAD.txt was removed
CORPUSX_OBESIDAD.txt was removed
CLASESX_OBESIDAD.txt was removed
sw_cpp_OBESIDAD.exe was removed
nb-n1...fit
nb-n1...predict
CREGEX...predict
fold: 3
CURVE: RESULTS
MODEL: nb-n1-cregex
PATH-fregex... G:\My Drive\Research\